## Introduction

This notebook builds a content-based recommendation system for Steam games. The goal is to recommend games to users based on their preferences, using game metadata and user review data. We'll connect to a MariaDB database, process the data, engineer features, and use text similarity and popularity to generate recommendations.

## Installing MariaDB Dependencies

Before we can connect to the MariaDB database, we need to ensure the required system libraries are installed. The following code installs the MariaDB development library using the system's package manager. This is necessary for Python packages to communicate with MariaDB.


In [ ]:
# For Important MariaDB Connector
import subprocess
import sys
def install_dep_mariadb():
    """
    This is for installing maria db lib using terminal
    """
    command = ['sudo', 'apt-get', 'install', 'libmariadb-dev']
    try:
        print(f"Executing command: {' '.join(command)}")
        result = subprocess.run(command, check=True, capture_output=True, text=True)
        print(result.stdout)
    except Exception as e:
        print(f"I Don't know some error ig {e}")
install_dep_mariadb()

## Installing Python Packages

We need several Python libraries for data processing, machine learning, and database connectivity. This cell installs `pandas`, `sqlalchemy`, `scikit-learn`, and `mariadb` so we can proceed with data loading and analysis.


In [ ]:
# Install required packages (for notebook environments)
!pip install pandas sqlalchemy scikit-learn mariadb

## Importing Libraries and Setting Up Database Connection

Here, we import all the necessary Python libraries for feature engineering, database connection, and machine learning. We also securely retrieve database credentials using Kaggle's secrets manager. The SQL query aggregates game metadata, genres, categories, and tags from multiple tables, filtering for games with enough reviews to be meaningful.


In [ ]:
# Import necessary libraries for feature engineering, database connection, and ML
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import pickle
import pandas as pd
from sqlalchemy import create_engine
import mariadb
from kaggle_secrets import UserSecretsClient
import joblib

# Retrieve database credentials securely from Kaggle secrets
user_secrets = UserSecretsClient()
DB_HOST = user_secrets.get_secret("DB_HOST")
DB_NAME = user_secrets.get_secret("DB_NAME")
DB_PASSWORD = user_secrets.get_secret("DB_PASSWORD")
DB_PORT = user_secrets.get_secret("DB_PORT")
DB_USER = user_secrets.get_secret("DB_USER")

# SQL query to fetch game data and aggregate genres, categories, and tags
# Here I used above than 100 due to the limitations of processing provided by kaggle
sql_query = """
SELECT
    a.id, a.name, a.short_description, a.positive_reviews, a.negative_reviews, a.achievements_count, a.recommendations,
    GROUP_CONCAT(DISTINCT g.name SEPARATOR ' ') AS genres,
    GROUP_CONCAT(DISTINCT c.name SEPARATOR ' ') AS categories,
    GROUP_CONCAT(DISTINCT t.name SEPARATOR ' ') AS tags
FROM
    apps a
LEFT JOIN app_genres ag ON a.id = ag.app_id
LEFT JOIN genres g ON ag.genre_id = g.id
LEFT JOIN app_categories ac ON a.id = ac.app_id
LEFT JOIN categories c ON ac.category_id = c.id
LEFT JOIN app_tags atg ON a.id = atg.app_id
LEFT JOIN tags t ON atg.tag_id = t.id
WHERE
    a.type = 'game' AND a.positive_reviews + a.negative_reviews > 100
GROUP BY
    a.id;
"""

# Create SQLAlchemy engine and load data into DataFrame
engine = create_engine(
    f"mariadb+mariadbconnector://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
)
dfMain = pd.read_sql(sql_query, engine)

## Loading and Preparing Data and feature Engineering

We load the game data from the database into a pandas DataFrame. To prepare for feature engineering, we fill missing values and ensure numeric columns are properly typed. This step is crucial for robust downstream processing. To compare games based on their content, we combine genres, categories, tags, and descriptions into a single text feature called "soup". We also add keywords to represent achievement-heavy games and those with overwhelmingly positive reviews. This enriched text representation helps our recommendation engine understand both content and sentiment.


In [ ]:
# Prepare DataFrame for feature engineering
df = dfMain.copy()
df.fillna('', inplace=True)
df.head()

# Convert review and achievement columns to integers, fill missing values
for col in ['positive_reviews', 'negative_reviews', 'achievements_count', 'recommendations']:
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)

# Ensure text columns are filled with empty strings if missing
for col in ['genres', 'categories', 'tags', 'short_description']:
    df[col] = df[col].fillna("")

def create_soup(x):
    text_features = f"{x['genres']} {x['categories']} {x['tags']} {x['short_description']}"
    achievement_keywords = ' '.join(['achievementheavy'] * (x['achievements_count'] // 20))
    sentiment_keywords = ''
    total_reviews = x['positive_reviews'] + x['negative_reviews']
    if total_reviews > 20 and (x['positive_reviews'] / total_reviews) > 0.80:
        sentiment_keywords = ' '.join(['overwhelminglypositive'] * 5)
    return f"{text_features} {achievement_keywords} {sentiment_keywords}"

# Create a 'soup' feature for each game for text-based similarity
df['soup'] = df.apply(create_soup, axis=1)

## Text Vectorization and Similarity Calculation

We use `CountVectorizer` to convert the "soup" text into a matrix of token counts, ignoring common English stop words. Then, we compute cosine similarity between all games, which quantifies how similar each game is to every other based on their metadata and descriptions. We also scale the popularity score for each game, which will help balance recommendations between popular and niche titles.


In [ ]:
# Vectorize the 'soup' feature, removing common English stop words
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df['soup'])

# Compute cosine similarity between all games
cosine_sim = cosine_similarity(count_matrix, count_matrix)

# Map game IDs to DataFrame indices for fast lookup
id_to_index = pd.Series(df.index, index=df['id']).to_dict()

# Scale popularity score for use in recommendations
scaler = MinMaxScaler()
df['popularity_score'] = scaler.fit_transform(df[['recommendations']])

number_of_game_recommendations = 20

## Recommendation Function

This function takes a list of user-preferred games and computes recommendations. It combines similarity scores, user preferences (like or opposite), and popularity to rank games. The function excludes the input games from the results and returns the top recommendations.


In [ ]:
def get_recommendations(input_games: list, niche_factor: float, cosine_sim, dataframe, id_map):
    """
    Given a list of input games, return top recommendations based on similarity and popularity.
    Supports boosting or reducing scores based on user preference (like/opposite).
    """
    total_scores = pd.Series(0.0, index = dataframe.index)
    input_game_indices = []

    for game in input_games:
        game_id = game['id']
        multiplier = game.get('multiplier', 1.0)
        pref_type = game.get('type', 'like')

        if game_id not in id_map:
            print(f"Warning: Game with ID '{game_id}' not found. Skipping.")
            continue

        idx = id_map[game_id]
        input_game_indices.append(idx)

        sim_scores = cosine_sim[idx]
        adjusted_scores = -sim_scores if pref_type == "opposite" else sim_scores
        total_scores += (adjusted_scores * multiplier)

    # Adjust scores by popularity and niche factor
    adjustment = 1 + (dataframe['popularity_score'] * niche_factor)
    total_scores = total_scores * adjustment

    # Exclude input games from recommendations
    total_scores = total_scores.drop(input_game_indices, errors='ignore')
    top_10_indices = total_scores.sort_values(ascending=False).head(10).index

    return dataframe['name'].iloc[top_10_indices]

# Example user input: list of games with preferences
user_input_by_id = [
    {'id': 25, 'multiplier': 2.0, 'type': 'like'},      # Cyberpunk 2077
    {'id': 1245620, 'multiplier': 1.0, 'type': 'like'}, # ELDEN RING
    # {'id': 1349130, 'multiplier': 1.0, 'type': 'like'}  # Gollum (commented out)
]

# Get recommendations and print results
popular_recs = get_recommendations(user_input_by_id, 1.0, cosine_sim, df, id_to_index)
print("\n" + "="*50)
print("RECOMMENDATIONS (POPULARITY MODE):")
print(popular_recs)
print("="*50)


## Saving Model Artifacts

Finally, we save the computed similarity matrix, processed DataFrame, and ID-to-index mapping as pickle files. This allows us to reuse these artifacts later without recomputing, making the recommendation system efficient and scalable.


In [ ]:
with open('cosine_sim.pkl', 'wb') as f:
    pickle.dump(cosine_sim, f)
with open('dataframe.pkl', 'wb') as f:
    pickle.dump(df, f)
with open('id_to_index.pkl', 'wb') as f:
    pickle.dump(id_to_index, f)